In [113]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Flatten, Input, Conv2D, BatchNormalization, MaxPooling2D
from keras.optimizers import Adam
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np

In [114]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [115]:
n_classes = 10
batch_size = 64
input_shape = x_train.shape[1], x_train.shape[2], 1

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

print('Number of classes:', n_classes)
print('Number of train samples:', x_train.shape[0])
print('Number of test samples:', x_test.shape[0])

Number of classes: 10
Number of train samples: 60000
Number of test samples: 10000


In [116]:
input = Input(shape=input_shape)

x = Conv2D(8, 3, activation='relu')(input)
x = MaxPooling2D()(x)
x = BatchNormalization()(x)

x = Conv2D(16, 3, activation='relu')(x)
x = MaxPooling2D()(x)
x = BatchNormalization()(x)

x = Flatten()(x)

x = Dense(n_classes, activation='softmax')(x)

model = Model(input, x)
model.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_14 (Conv2D)          (None, 26, 26, 8)         80        
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 13, 13, 8)        0         
 g2D)                                                            
                                                                 
 batch_normalization_2 (Batc  (None, 13, 13, 8)        32        
 hNormalization)                                                 
                                                                 
 conv2d_15 (Conv2D)          (None, 11, 11, 16)        1168      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 5, 5, 16)         0  

In [117]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [118]:
callbacks = [
    EarlyStopping(patience=3, monitor='val_loss', mode="auto"),
    ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.0001, monitor='val_loss', mode='auto', verbose=1),
    ModelCheckpoint('best_model.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='auto', verbose=1)
]

In [119]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=5, validation_split=0.2, callbacks=callbacks)

Epoch 1/5
748/750 [============================>.] - ETA: 0s - loss: 0.2592 - accuracy: 0.9234
Epoch 1: val_loss improved from inf to 0.10101, saving model to best_model.h5
750/750 [==============================] - 23s 29ms/step - loss: 0.2587 - accuracy: 0.9234 - val_loss: 0.1010 - val_accuracy: 0.9718 - lr: 0.0010
Epoch 2/5
748/750 [============================>.] - ETA: 0s - loss: 0.0770 - accuracy: 0.9761
Epoch 2: val_loss improved from 0.10101 to 0.07214, saving model to best_model.h5
750/750 [==============================] - 20s 27ms/step - loss: 0.0770 - accuracy: 0.9761 - val_loss: 0.0721 - val_accuracy: 0.9790 - lr: 0.0010
Epoch 3/5
750/750 [==============================] - ETA: 0s - loss: 0.0562 - accuracy: 0.9824
Epoch 3: val_loss improved from 0.07214 to 0.05819, saving model to best_model.h5
750/750 [==============================] - 20s 27ms/step - loss: 0.0562 - accuracy: 0.9824 - val_loss: 0.0582 - val_accuracy: 0.9828 - lr: 0.0010
Epoch 4/5
750/750 [================

In [120]:
model.load_weights('best_model.h5')

In [121]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.051496416330337524
Test accuracy: 0.9837999939918518
